In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Installing Libraries (if any)
###Drive Link--->(https://drive.google.com/drive/folders/1lMVAN3glfGllGXQzQODKoH5U_QdEQ3n-?usp=sharing)

In [ ]:
!pip install pytorch_lightning
!pip install torchattacks
!pip install pyttsx3
!sudo apt-get install espeak
!pip install gTTS
!pip install textblob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 826.4/826.4 KB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.6/518.6 KB 40.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.9/155.9 KB 4.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  espeak-data libespeak1 libportaudio2 libsonic0
The following NEW packages will be installed:
  espeak espeak-data libespeak1 libportaudio2 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 19 not upgraded.
Need to get 1,220 kB of archives.
After this operation, 3,060 kB of additional disk space will b

# Importing Libraries 
###Drive Link--->(https://drive.google.com/drive/folders/1lMVAN3glfGllGXQzQODKoH5U_QdEQ3n-?usp=sharing)

In [ ]:
import os
import torch
from torch.utils.data import Dataset, random_split, DataLoader
import pytorch_lightning as pl
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
from torchvision import models
import PIL
from tqdm import tqdm
import cv2
import glob
import torchattacks as adv
import json
import csv
from gtts import gTTS

#Question-1
###Drive Link--->(https://drive.google.com/drive/folders/1lMVAN3glfGllGXQzQODKoH5U_QdEQ3n-?usp=sharing)

## Utility Functions and Classes

In [ ]:
class CNN_3Layer(pl.LightningModule):
  
  def __init__(self, num_classes):
      super().__init__()
      self.model = nn.Sequential(
          nn.Conv2d(3,4,5),
          nn.Conv2d(4,5,4),
          nn.Conv2d(5,6,3),
          nn.Flatten(),
          nn.Linear(3174,1024),
          nn.Linear(1024,num_classes)
      )
      self.loss = nn.CrossEntropyLoss()
  
  def forward(self, x):
      return self.model(x)
  
  def training_step(self, batch, batch_no):
      x, y = batch
      logits = self(x)
      loss = self.loss(logits, y)
      return loss
  
  def configure_optimizers(self):
      return torch.optim.Adam(self.parameters(),lr=0.001)

In [ ]:
class Model(pl.LightningModule):

  def __init__(self, model_name, num_classes):
    super().__init__()
    self.model_name = model_name
    if self.model_name=='resnet_34':
      self.model = models.resnet34(weights=None)
      self.features = self.model.fc.in_features 
      self.model.fc = nn.Linear(self.features,num_classes)
    if self.model_name=='densenet_121':
      self.model = models.densenet121(weights=None)
      self.features = self.model.classifier.in_features
      self.model.classifier = nn.Linear(self.features,num_classes)
    self.loss = nn.CrossEntropyLoss()

  def forward(self, x):
    return self.model(x)

  def training_step(self, batch, batch_no):
    x, y = batch
    logits = self(x)
    loss = self.loss(logits, y)
    return loss

  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(),lr=0.001)

In [ ]:
class Detector_Train(pl.LightningModule):

  def __init__(self, model_name, attack):
    super().__init__()
    self.model_name = model_name
    self.attack = attack
    if self.model_name=='resnet_34':
      self.model = models.resnet34(weights=None)
      self.features = self.model.fc.in_features 
      self.model.fc = nn.Linear(self.features,2)
    if self.model_name=='densenet_121':
      self.model = models.densenet121(weights=None)
      self.features = self.model.classifier.in_features
      self.model.classifier = nn.Linear(self.features,2)
    self.loss = nn.CrossEntropyLoss()

  def forward(self, x, y):
    return self.model(x), self.model(self.attack(x,y))

  def training_step(self, batch, batch_no):
    x, y = batch
    logits_clean, logits_attack = self(x,y)
    loss = self.loss(logits_clean, torch.tensor([0]*logits_clean.shape[0]).cuda())+self.loss(logits_attack, torch.tensor([1]*logits_attack.shape[0]).cuda())
    return loss

  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(),lr=0.001)

In [ ]:
def transform_fn(img, mode='Train', resize=False):
  if resize==True:
    img = img.resize((224,224))
  tensor = transforms.ToTensor()(img)
  return tensor

  #Comment Out, if performing Normalization on CIFAR10
  '''if mode=='Train':
    return transforms.Normalize([0.4914, 0.4822, 0.4465],[0.2454, 0.2419, 0.2601])(tensor)
  else:
    return transforms.Normalize([0.4986, 0.4843, 0.4453],[0.2438, 0.2403, 0.2571])(tensor)'''

def model_return(ckpt_path, model_obj):
  ckpt = torch.load(ckpt_path)
  model_obj.load_state_dict(ckpt['state_dict'])
  return model_obj.eval().cuda()

def FGSM(img, gradient, epsilon=0.01):
  pertubation = img + gradient.sign()*epsilon
  return pertubation

def attack(test_loader, model_class, attack_func, eps):
  correct_per = 0
  correct_no = 0
  for i,j in test_loader:
    i,j = i.cuda(), j.cuda()
    i.requires_grad = True
    prob_no = model_class(i)
    out_no = prob_no.max(1, keepdim=True)[1]
    if out_no.item() != j.item():
      continue
    else:
      correct_no += 1
      loss_no = nn.CrossEntropyLoss()(prob_no, j)
      model_class.zero_grad()
      loss_no.backward()
      gradient = i.grad.data
      i_perturb = attack_func(i, gradient, eps)
      prob_perturb = model_class(i_perturb)
      out_perturb = prob_perturb.max(1, keepdim=True)[1]
      if out_perturb.item() == j.item():
        correct_per += 1
  final_acc_per = correct_per/len(test_loader)
  final_acc_no = correct_no/len(test_loader)
  print("Original Accuracy ===> "+str(100*final_acc_no)+"%")
  print("Accuracy at epsilon= "+str(eps)+" ===> "+str(100*final_acc_per)+"%")

def prebuilt_detect(test_loader, model_class, attack_func, epsilon):
  correct_per = 0
  correct_no = 0
  for i,j in test_loader:
    i,j = i.cuda(), j.cuda()
    prob_no = model_class(i)
    out_no = prob_no.max(1, keepdim=True)[1]
    if out_no.item() == 0.0:
      correct_no += 1
    i_perturb = attack_func(i,j)
    prob_perturb = model_class(i_perturb)
    out_perturb = prob_perturb.max(1, keepdim=True)[1]
    if out_perturb.item() == 1.0:
      correct_per += 1
  final_acc_per = correct_per/len(test_loader)
  final_acc_no = correct_no/len(test_loader)
  print("Clean Accuracy ===> "+str(100*final_acc_no)+"%")
  print("Attack Accuracy at epsilon= "+str(epsilon)+" ===> "+str(100*final_acc_per)+"%")

def prebuilt_attack(test_loader, model_class, attack_func, epsilon):
  correct_per = 0
  correct_no = 0
  attack = attack_func(model_class, eps=epsilon)
  for i,j in test_loader:
    i,j = i.cuda(), j.cuda()
    prob_no = model_class(i)
    out_no = prob_no.max(1, keepdim=True)[1]
    if out_no.item() != j.item():
      continue
    else:
      correct_no += 1
      i_perturb = attack(i,j)
      prob_perturb = model_class(i_perturb)
      out_perturb = prob_perturb.max(1, keepdim=True)[1]
      if out_perturb.item() == j.item():
        correct_per += 1
  final_acc_per = correct_per/len(test_loader)
  final_acc_no = correct_no/len(test_loader)
  print("Original Accuracy ===> "+str(100*final_acc_no)+"%")
  print("Accuracy at epsilon= "+str(epsilon)+" ===> "+str(100*final_acc_per)+"%")

##(i)

In [ ]:
#train_CIFAR10 = torch.utils.data.DataLoader(dataset=datasets.CIFAR10('../data', train=True, download=True, transform=transform_fn),batch_size=16, shuffle=True, pin_memory=True, num_workers=2)
test_CIFAR10 = torch.utils.data.DataLoader(dataset=datasets.CIFAR10('../data', train=False, download=True, transform=transform_fn),batch_size=1, shuffle=True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ../data/cifar-10-python.tar.gz to ../data


In [ ]:
Model_Scratch = CNN_3Layer(num_classes=10).train().cuda()
trainer = pl.Trainer(accelerator='gpu', max_epochs=20, default_root_dir='/content/drive/MyDrive/DAI_Assignment_1/Q1/3_Layer_CNN', benchmark=True)
trainer.fit(Model_Scratch, train_CIFAR10)
del trainer, Model_Scratch

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | Sequential       | 3.3 M 
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
3.3 M     Trainable params
0         Non-trainable params
3.3 M     Total params
13.049    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


In [ ]:
Model_Test = model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/3_Layer_CNN/lightning_logs/version_1/checkpoints/epoch=19-step=62500.ckpt', CNN_3Layer(num_classes=10))

In [ ]:
attack(test_CIFAR10, Model_Test, FGSM, 0.001)

Original Accuracy ===> 37.14%
Accuracy at epsilon= 0.001 ===> 23.48%


In [ ]:
attack(test_CIFAR10, Model_Test, FGSM, 0.002)

Original Accuracy ===> 37.14%
Accuracy at epsilon= 0.002 ===> 13.420000000000002%


In [ ]:
attack(test_CIFAR10, Model_Test, FGSM, 0.004)

Original Accuracy ===> 37.14%
Accuracy at epsilon= 0.004 ===> 3.6700000000000004%


In [ ]:
attack(test_CIFAR10, Model_Test, FGSM, 0.008)

Original Accuracy ===> 37.14%
Accuracy at epsilon= 0.008 ===> 0.36%


In [ ]:
attack(test_CIFAR10, Model_Test, FGSM, 0.01)

Original Accuracy ===> 37.14%
Accuracy at epsilon= 0.01 ===> 0.13%


##(ii)

In [ ]:
train_dataset = datasets.SVHN(root='./data', split='train', download=True, transform=transforms.ToTensor())
test_dataset = datasets.SVHN(root='./data', split='test', download=True, transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=len(train_dataset))
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=len(test_dataset))

train_data = next(iter(train_loader))
train_mean = train_data[0].mean()
train_std = train_data[0].std()

test_data = next(iter(test_loader))
test_mean = test_data[0].mean()
test_std = test_data[0].std()

  0%|          | 0/182040794 [00:00<?, ?it/s]

  0%|          | 0/64275384 [00:00<?, ?it/s]

In [ ]:
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((train_mean.item(),), (train_std.item(),)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((test_mean.item(),), (test_std.item(),)),
])

In [ ]:
train_SVHN = torch.utils.data.DataLoader(dataset=datasets.SVHN('./data', split='train', download=True, transform=transform_train),batch_size=16, shuffle=True, pin_memory=True, num_workers=2)
test_SVHN = torch.utils.data.DataLoader(dataset=datasets.SVHN('./data', split='test', download=True, transform=transform_test),batch_size=1, shuffle=True)

Using downloaded and verified file: ./data/test_32x32.mat


In [ ]:
resnet = Model('resnet_34',10).train().cuda()
trainer = pl.Trainer(accelerator='gpu', max_epochs=20, default_root_dir='/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34', benchmark=True)
trainer.fit(resnet, train_CIFAR10)
del trainer, resnet

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | ResNet           | 21.3 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
21.3 M    Trainable params
0         Non-trainable params
21.3 M    Total params
85.159    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


In [ ]:
resnet = Model('resnet_34',10).train().cuda()
trainer = pl.Trainer(accelerator='gpu', max_epochs=20, resume_from_checkpoint='/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34/lightning_logs/version_2/checkpoints/epoch=11-step=54948.ckpt', default_root_dir='/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34', benchmark=True)
trainer.fit(resnet, train_SVHN)
del trainer, resnet

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34/lightning_logs/version_2/checkpoints/epoch=11-step=54948.ckpt
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:338: UserWarning: The dirpath has changed from '/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34/lightning_logs/version_2/checkpoints' to '/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34/lightning_logs/version_3/checkpoints', therefore `best_model_score`, `kth_best_model_path`, `kth_value`, `last_model_path` and `best_k_models` won't be reloaded. Only `best_model_path` will be 

Training: 4579it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


In [ ]:
densenet = Model('densenet_121',10).train().cuda()
trainer = pl.Trainer(accelerator='gpu', max_epochs=20, default_root_dir='/content/drive/MyDrive/DAI_Assignment_1/Q1/densenet121', benchmark=True)
trainer.fit(densenet, train_CIFAR10)
del trainer, densenet

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | DenseNet         | 7.0 M 
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
7.0 M     Trainable params
0         Non-trainable params
7.0 M     Total params
27.856    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


In [ ]:
densenet = Model('densenet_121',10).train().cuda()
trainer = pl.Trainer(accelerator='gpu', max_epochs=20, default_root_dir='/content/drive/MyDrive/DAI_Assignment_1/Q1/densenet121', benchmark=True)
trainer.fit(densenet, train_SVHN)
del trainer, densenet

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | DenseNet         | 7.0 M 
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
7.0 M     Trainable params
0         Non-trainable params
7.0 M     Total params
27.856    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


### Testing on CIFAR10 with FGSM

In [ ]:
Model_Resnet = model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34/lightning_logs/version_1/checkpoints/epoch=19-step=62500.ckpt', Model('resnet_34',num_classes=10))

In [ ]:
attack(test_CIFAR10, Model_Resnet, FGSM, 0.001)

Original Accuracy ===> 32.24%
Accuracy at epsilon= 0.001 ===> 28.549999999999997%


In [ ]:
attack(test_CIFAR10, Model_Resnet, FGSM, 0.002)

Original Accuracy ===> 32.24%
Accuracy at epsilon= 0.002 ===> 25.36%


In [ ]:
attack(test_CIFAR10, Model_Resnet, FGSM, 0.004)

Original Accuracy ===> 32.24%
Accuracy at epsilon= 0.004 ===> 20.11%


In [ ]:
attack(test_CIFAR10, Model_Resnet, FGSM, 0.008)

Original Accuracy ===> 32.24%
Accuracy at epsilon= 0.008 ===> 12.870000000000001%


In [ ]:
attack(test_CIFAR10, Model_Resnet, FGSM, 0.01)

Original Accuracy ===> 32.24%
Accuracy at epsilon= 0.01 ===> 10.43%


In [ ]:
Model_Densenet = model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/densenet121/lightning_logs/version_0/checkpoints/epoch=19-step=62500.ckpt', Model('densenet_121',num_classes=10))

In [ ]:
attack(test_CIFAR10, Model_Densenet, FGSM, 0.001)

Original Accuracy ===> 78.78%
Accuracy at epsilon= 0.001 ===> 72.78999999999999%


In [ ]:
attack(test_CIFAR10, Model_Densenet, FGSM, 0.002)

Original Accuracy ===> 78.78%
Accuracy at epsilon= 0.002 ===> 66.07%


In [ ]:
attack(test_CIFAR10, Model_Densenet, FGSM, 0.004)

Original Accuracy ===> 78.78%
Accuracy at epsilon= 0.004 ===> 53.14%


In [ ]:
attack(test_CIFAR10, Model_Densenet, FGSM, 0.008)

Original Accuracy ===> 78.78%
Accuracy at epsilon= 0.008 ===> 32.98%


In [ ]:
attack(test_CIFAR10, Model_Densenet, FGSM, 0.01)

Original Accuracy ===> 78.78%
Accuracy at epsilon= 0.01 ===> 26.08%


### Testing on SVHN with FGSM

In [ ]:
Model_Resnet = model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34/lightning_logs/version_3/checkpoints/epoch=19-step=91580.ckpt', Model('resnet_34',num_classes=10))

In [ ]:
attack(test_SVHN, Model_Resnet, FGSM, 0.001)

Original Accuracy ===> 93.46957590657652%
Accuracy at epsilon= 0.001 ===> 92.79348494161033%


In [ ]:
attack(test_SVHN, Model_Resnet, FGSM, 0.002)

Original Accuracy ===> 93.46957590657652%
Accuracy at epsilon= 0.002 ===> 92.0059926244622%


In [ ]:
attack(test_SVHN, Model_Resnet, FGSM, 0.004)

Original Accuracy ===> 93.46957590657652%
Accuracy at epsilon= 0.004 ===> 90.53088506453597%


In [ ]:
attack(test_SVHN, Model_Resnet, FGSM, 0.008)

Original Accuracy ===> 93.46957590657652%
Accuracy at epsilon= 0.008 ===> 87.08128457283344%


In [ ]:
attack(test_SVHN, Model_Resnet, FGSM, 0.01)

Original Accuracy ===> 93.46957590657652%
Accuracy at epsilon= 0.01 ===> 85.2681315304241%


In [ ]:
Model_Densenet = model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/densenet121/lightning_logs/version_1/checkpoints/epoch=19-step=91580.ckpt', Model('densenet_121',num_classes=10))

In [ ]:
attack(test_SVHN, Model_Densenet, FGSM, 0.001)

Original Accuracy ===> 93.68853718500307%
Accuracy at epsilon= 0.001 ===> 92.99323909035033%


In [ ]:
attack(test_SVHN, Model_Densenet, FGSM, 0.002)

Original Accuracy ===> 93.68853718500307%
Accuracy at epsilon= 0.002 ===> 92.22111247695143%


In [ ]:
attack(test_SVHN, Model_Densenet, FGSM, 0.004)

Original Accuracy ===> 93.68853718500307%
Accuracy at epsilon= 0.004 ===> 90.5539336201598%


In [ ]:
attack(test_SVHN, Model_Densenet, FGSM, 0.008)

Original Accuracy ===> 93.68853718500307%
Accuracy at epsilon= 0.008 ===> 86.47433927473878%


In [ ]:
attack(test_SVHN, Model_Densenet, FGSM, 0.01)

Original Accuracy ===> 93.68853718500307%
Accuracy at epsilon= 0.01 ===> 84.3577135832821%


### Testing on SVHN with PGD & Jitter

In [ ]:
Model_Resnet = model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34/lightning_logs/version_3/checkpoints/epoch=19-step=91580.ckpt', Model('resnet_34',num_classes=10))

In [ ]:
prebuilt_attack(test_SVHN, Model_Resnet, adv.PGD, 0.005)

Original Accuracy ===> 93.46957590657652%
Accuracy at epsilon= 0.005 ===> 62.20421020282729%


In [ ]:
prebuilt_attack(test_SVHN, Model_Resnet, adv.Jitter, 0.005)

Original Accuracy ===> 93.46957590657652%
Accuracy at epsilon= 0.005 ===> 62.146588813767664%


In [ ]:
Model_Densenet = model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/densenet121/lightning_logs/version_1/checkpoints/epoch=19-step=91580.ckpt', Model('densenet_121',num_classes=10))

In [ ]:
prebuilt_attack(test_SVHN, Model_Densenet, adv.PGD, 0.005)

Original Accuracy ===> 93.68853718500307%
Accuracy at epsilon= 0.005 ===> 64.10571604179471%


In [ ]:
prebuilt_attack(test_SVHN, Model_Densenet, adv.Jitter, 0.005)

Original Accuracy ===> 93.68853718500307%
Accuracy at epsilon= 0.005 ===> 64.01736324523664%


##(iii)

### Distinguishing between Clean and PGD via Resnet34 (SVHN)

In [ ]:
del attack_res, detect_res
attack_res = adv.PGD(model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34/lightning_logs/version_3/checkpoints/epoch=19-step=91580.ckpt', Model('resnet_34',num_classes=10)),eps=0.005)
detect_res = Detector_Train('resnet_34', attack_res)

In [ ]:
trainer = pl.Trainer(accelerator='gpu', max_epochs=5, default_root_dir='/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34_detector', benchmark=True)
trainer.fit(detect_res, train_SVHN)
del trainer, detect_res, attack_res

In [ ]:
torch.backends.cudnn.benchmark = True
attack_res = adv.PGD(model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34/lightning_logs/version_3/checkpoints/epoch=19-step=91580.ckpt', Model('resnet_34',num_classes=10)),eps=0.005)
Model_Resnet = model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34_detector/lightning_logs/version_1/checkpoints/epoch=4-step=22895.ckpt', Model('resnet_34',num_classes=2))

In [ ]:
prebuilt_detect(test_SVHN, Model_Resnet, attack_res, 0.005)
del Model_Resnet, attack_res

Clean Accuracy ===> 0.18822987092808852%
Attack Accuracy at epsilon= 0.005 ===> 100.0%


### Distinguishing between Clean and Jitter via Resnet34 (SVHN)

In [ ]:
attack_res = adv.Jitter(model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34/lightning_logs/version_3/checkpoints/epoch=19-step=91580.ckpt', Model('resnet_34',num_classes=10)),eps=0.005)
detect_res = Detector_Train('resnet_34', attack_res)

In [ ]:
trainer = pl.Trainer(accelerator='gpu', max_epochs=5, resume_from_checkpoint = '/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34_detector/lightning_logs/version_3/checkpoints/epoch=3-step=18316.ckpt', default_root_dir='/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34_detector', benchmark=True)
trainer.fit(detect_res, train_SVHN)
del trainer, detect_res, attack_res

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:55: LightningDeprecationWarning: Setting `Trainer(resume_from_checkpoint=)` is deprecated in v1.5 and will be removed in v2.0. Please pass `Trainer.fit(ckpt_path=)` directly instead.
  rank_zero_deprecation(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/trainer.py:1906: LightningDeprecationWarning: `trainer.resume_from_checkpoint` is deprecated in v1.5 and will be removed in v2.0. Specify the fit checkpoint path with `trainer.fit(ckpt_path=)` instead.
  rank_zero_deprecation(
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpo

Training: 4579it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


In [ ]:
attack_res = adv.Jitter(model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34/lightning_logs/version_3/checkpoints/epoch=19-step=91580.ckpt', Model('resnet_34',num_classes=10)),eps=0.005)
Model_Resnet = model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34_detector/lightning_logs/version_4/checkpoints/epoch=4-step=22895.ckpt', Model('resnet_34',num_classes=2))
prebuilt_detect(test_SVHN, Model_Resnet, attack_res, 0.005)
del Model_Resnet, attack_res

Clean Accuracy ===> 73.5479409956976%
Attack Accuracy at epsilon= 0.005 ===> 0.0%


### Distinguishing between Clean and FGSM via Resnet34 (SVHN)

In [ ]:
attack_res = adv.FGSM(model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34/lightning_logs/version_3/checkpoints/epoch=19-step=91580.ckpt', Model('resnet_34',num_classes=10)),eps=0.005)
detect_res = Detector_Train('resnet_34', attack_res)

In [ ]:
trainer = pl.Trainer(accelerator='gpu', max_epochs=5, default_root_dir='/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34_detector', benchmark=True)
trainer.fit(detect_res, train_SVHN)
del trainer, detect_res, attack_res

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | ResNet           | 21.3 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
21.3 M    Trainable params
0         Non-trainable params
21.3 M    Total params
85.143    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


In [ ]:
attack_res = adv.FGSM(model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34/lightning_logs/version_3/checkpoints/epoch=19-step=91580.ckpt', Model('resnet_34',num_classes=10)),eps=0.005)
Model_Resnet = model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34_detector/lightning_logs/version_5/checkpoints/epoch=4-step=22895.ckpt', Model('resnet_34',num_classes=2))
prebuilt_detect(test_SVHN, Model_Resnet, attack_res, 0.005)
del Model_Resnet, attack_res

Clean Accuracy ===> 71.7040565457898%
Attack Accuracy at epsilon= 0.005 ===> 0.0%


### Distinguishing between Clean and PGD via Resnet34 (CIFAR10)

In [ ]:
attack_res = adv.PGD(model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34/lightning_logs/version_1/checkpoints/epoch=19-step=62500.ckpt', Model('resnet_34',num_classes=10)),eps=0.005)
detect_res = Detector_Train('resnet_34', attack_res)

In [ ]:
trainer = pl.Trainer(accelerator='gpu', max_epochs=5, default_root_dir='/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34_detector_cifar', benchmark=True)
trainer.fit(detect_res, train_CIFAR10)
del trainer, detect_res, attack_res

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | ResNet           | 21.3 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
21.3 M    Trainable params
0         Non-trainable params
21.3 M    Total params
85.143    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


In [ ]:
attack_res = adv.PGD(model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34/lightning_logs/version_1/checkpoints/epoch=19-step=62500.ckpt', Model('resnet_34',num_classes=10)),eps=0.005)
Model_Resnet = model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34_detector_cifar/lightning_logs/version_3/checkpoints/epoch=4-step=15625.ckpt', Model('resnet_34',num_classes=2))
prebuilt_detect(test_CIFAR10, Model_Resnet, attack_res, 0.005)
del Model_Resnet, attack_res

Clean Accuracy ===> 3.36%
Attack Accuracy at epsilon= 0.005 ===> 96.64%


### Distinguishing between Clean and Jitter via Resnet34 (CIFAR10)

In [ ]:
attack_res = adv.Jitter(model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34/lightning_logs/version_1/checkpoints/epoch=19-step=62500.ckpt', Model('resnet_34',num_classes=10)),eps=0.005)
detect_res = Detector_Train('resnet_34', attack_res)

In [ ]:
trainer = pl.Trainer(accelerator='gpu', max_epochs=5, default_root_dir='/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34_detector_cifar', benchmark=True)
trainer.fit(detect_res, train_CIFAR10)
del trainer, detect_res, attack_res

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | ResNet           | 21.3 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
21.3 M    Trainable params
0         Non-trainable params
21.3 M    Total params
85.143    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


In [ ]:
attack_res = adv.Jitter(model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34/lightning_logs/version_1/checkpoints/epoch=19-step=62500.ckpt', Model('resnet_34',num_classes=10)),eps=0.005)
Model_Resnet = model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34_detector_cifar/lightning_logs/version_4/checkpoints/epoch=4-step=15625.ckpt', Model('resnet_34',num_classes=2))
prebuilt_detect(test_CIFAR10, Model_Resnet, attack_res, 0.005)
del Model_Resnet, attack_res

Clean Accuracy ===> 17.119999999999997%
Attack Accuracy at epsilon= 0.005 ===> 82.89%


### Distinguishing between Clean and FGSM via Resnet34 (CIFAR10)

In [ ]:
attack_res = adv.FGSM(model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34/lightning_logs/version_1/checkpoints/epoch=19-step=62500.ckpt', Model('resnet_34',num_classes=10)),eps=0.005)
detect_res = Detector_Train('resnet_34', attack_res)

In [ ]:
trainer = pl.Trainer(accelerator='gpu', max_epochs=5, default_root_dir='/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34_detector_cifar', benchmark=True)
trainer.fit(detect_res, train_CIFAR10)
del trainer, detect_res, attack_res

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | ResNet           | 21.3 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
21.3 M    Trainable params
0         Non-trainable params
21.3 M    Total params
85.143    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


In [ ]:
attack_res = adv.FGSM(model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34/lightning_logs/version_1/checkpoints/epoch=19-step=62500.ckpt', Model('resnet_34',num_classes=10)),eps=0.005)
Model_Resnet = model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34_detector_cifar/lightning_logs/version_2/checkpoints/epoch=4-step=15625.ckpt', Model('resnet_34',num_classes=2))
prebuilt_detect(test_CIFAR10, Model_Resnet, attack_res, 0.005)
del Model_Resnet, attack_res

Clean Accuracy ===> 92.4%
Attack Accuracy at epsilon= 0.005 ===> 7.62%


### Cross-Attack Generalizability (Trained on FGSM and tested on PGD and Jitter)

In [ ]:
attack_res = adv.PGD(model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34/lightning_logs/version_1/checkpoints/epoch=19-step=62500.ckpt', Model('resnet_34',num_classes=10)),eps=0.005)
Model_Resnet = model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34_detector_cifar/lightning_logs/version_2/checkpoints/epoch=4-step=15625.ckpt', Model('resnet_34',num_classes=2))
prebuilt_detect(test_CIFAR10, Model_Resnet, attack_res, 0.005)
del Model_Resnet, attack_res

Clean Accuracy ===> 92.4%
Attack Accuracy at epsilon= 0.005 ===> 7.5600000000000005%


In [ ]:
attack_res = adv.Jitter(model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34/lightning_logs/version_1/checkpoints/epoch=19-step=62500.ckpt', Model('resnet_34',num_classes=10)),eps=0.005)
Model_Resnet = model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34_detector_cifar/lightning_logs/version_2/checkpoints/epoch=4-step=15625.ckpt', Model('resnet_34',num_classes=2))
prebuilt_detect(test_CIFAR10, Model_Resnet, attack_res, 0.005)
del Model_Resnet, attack_res

Clean Accuracy ===> 92.4%
Attack Accuracy at epsilon= 0.005 ===> 7.6%


### Cross-Attack Generalizability (Trained on Jitter and tested on PGD and FGSM)

In [ ]:
attack_res = adv.PGD(model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34/lightning_logs/version_1/checkpoints/epoch=19-step=62500.ckpt', Model('resnet_34',num_classes=10)),eps=0.005)
Model_Resnet = model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34_detector_cifar/lightning_logs/version_4/checkpoints/epoch=4-step=15625.ckpt', Model('resnet_34',num_classes=2))
prebuilt_detect(test_CIFAR10, Model_Resnet, attack_res, 0.005)
del Model_Resnet, attack_res

Clean Accuracy ===> 17.119999999999997%
Attack Accuracy at epsilon= 0.005 ===> 82.88%


In [ ]:
attack_res = adv.FGSM(model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34/lightning_logs/version_1/checkpoints/epoch=19-step=62500.ckpt', Model('resnet_34',num_classes=10)),eps=0.005)
Model_Resnet = model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34_detector_cifar/lightning_logs/version_4/checkpoints/epoch=4-step=15625.ckpt', Model('resnet_34',num_classes=2))
prebuilt_detect(test_CIFAR10, Model_Resnet, attack_res, 0.005)
del Model_Resnet, attack_res

Clean Accuracy ===> 17.119999999999997%
Attack Accuracy at epsilon= 0.005 ===> 82.91%


### Cross-Attack Generalizability (Trained on PGD and tested on Jitter and FGSM)

In [ ]:
attack_res = adv.Jitter(model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34/lightning_logs/version_1/checkpoints/epoch=19-step=62500.ckpt', Model('resnet_34',num_classes=10)),eps=0.005)
Model_Resnet = model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34_detector_cifar/lightning_logs/version_3/checkpoints/epoch=4-step=15625.ckpt', Model('resnet_34',num_classes=2))
prebuilt_detect(test_CIFAR10, Model_Resnet, attack_res, 0.005)
del Model_Resnet, attack_res

Clean Accuracy ===> 3.36%
Attack Accuracy at epsilon= 0.005 ===> 96.64%


In [ ]:
attack_res = adv.FGSM(model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34/lightning_logs/version_1/checkpoints/epoch=19-step=62500.ckpt', Model('resnet_34',num_classes=10)),eps=0.005)
Model_Resnet = model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34_detector_cifar/lightning_logs/version_3/checkpoints/epoch=4-step=15625.ckpt', Model('resnet_34',num_classes=2))
prebuilt_detect(test_CIFAR10, Model_Resnet, attack_res, 0.005)
del Model_Resnet, attack_res

Clean Accuracy ===> 3.36%
Attack Accuracy at epsilon= 0.005 ===> 96.63000000000001%


### Cross-Domain Generalizability (Trained on SVHN and tested on CIFAR10 with FGSM)

In [ ]:
attack_res = adv.FGSM(model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34/lightning_logs/version_1/checkpoints/epoch=19-step=62500.ckpt', Model('resnet_34',num_classes=10)),eps=0.005)
Model_Resnet = model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34_detector/lightning_logs/version_5/checkpoints/epoch=4-step=22895.ckpt', Model('resnet_34',num_classes=2))
prebuilt_detect(test_CIFAR10, Model_Resnet, attack_res, 0.005)
del Model_Resnet, attack_res

Clean Accuracy ===> 100.0%
Attack Accuracy at epsilon= 0.005 ===> 0.0%


### Cross-Domain Generalizability (Trained on SVHN and tested on CIFAR10 with PGD)

In [ ]:
attack_res = adv.PGD(model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34/lightning_logs/version_1/checkpoints/epoch=19-step=62500.ckpt', Model('resnet_34',num_classes=10)),eps=0.005)
Model_Resnet = model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34_detector/lightning_logs/version_5/checkpoints/epoch=4-step=22895.ckpt', Model('resnet_34',num_classes=2))
prebuilt_detect(test_CIFAR10, Model_Resnet, attack_res, 0.005)
del Model_Resnet, attack_res

Clean Accuracy ===> 100.0%
Attack Accuracy at epsilon= 0.005 ===> 0.0%


### Cross-Domain Generalizability (Trained on SVHN and tested on CIFAR10 with Jitter)

In [ ]:
attack_res = adv.Jitter(model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34/lightning_logs/version_1/checkpoints/epoch=19-step=62500.ckpt', Model('resnet_34',num_classes=10)),eps=0.005)
Model_Resnet = model_return('/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34_detector/lightning_logs/version_5/checkpoints/epoch=4-step=22895.ckpt', Model('resnet_34',num_classes=2))
prebuilt_detect(test_CIFAR10, Model_Resnet, attack_res, 0.005)
del Model_Resnet, attack_res

Clean Accuracy ===> 100.0%
Attack Accuracy at epsilon= 0.005 ===> 0.0%


#Question-2
###Drive Link--->(https://drive.google.com/drive/folders/1lMVAN3glfGllGXQzQODKoH5U_QdEQ3n-?usp=sharing)

##Utility Functions and Classes

In [ ]:
class Detector_Face(pl.LightningModule):

  def __init__(self, model_name):
    super().__init__()
    self.model_name = model_name
    if self.model_name=='resnet_34':
      self.model = models.resnet34(weights=None)
      self.features = self.model.fc.in_features 
      self.model.fc = nn.Linear(self.features,2)
    if self.model_name=='densenet_121':
      self.model = models.densenet121(weights=None)
      self.features = self.model.classifier.in_features
      self.model.classifier = nn.Linear(self.features,2)
    self.loss = nn.CrossEntropyLoss()

  def forward(self, x):
    return self.model(x)

  def training_step(self, batch, batch_no):
    x, y = batch
    logits = self(x)
    loss = self.loss(logits,y)
    return loss

  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(),lr=0.001)

In [ ]:
def flip_augment(path):
  list_images = os.listdir(path)
  path_images = [os.path.join(path, img_path) for img_path in list_images]
  count=0
  for img_path in tqdm(path_images):
    img = cv2.imread(img_path)
    img_out = cv2.flip(img, 1)
    cv2.imwrite(os.path.join(path, str(count)+'.jpg'),img_out)
    count+=1

def detect_eval(test_loader, model_class):
  correct = 0
  for i,j in test_loader:
    i,j = i.cuda(), j.cuda()
    prob = model_class(i)
    out = prob.max(1, keepdim=True)[1]
    if out.item() == j.item():
      correct += 1
  final_acc = correct/len(test_loader)
  print("Accuracy ===> "+str(100*final_acc)+"%")

## FaceSweep Preparations

In [ ]:
flip_augment('/content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real')

100%|██████████| 25/25 [00:07<00:00,  3.44it/s]


In [ ]:
!git clone https://github.com/kwanit1142/FaceSwap_DAI.git

Cloning into 'FaceSwap_DAI'...
remote: Enumerating objects: 174, done.
remote: Counting objects: 100% (174/174), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 174 (delta 79), reused 165 (delta 76), pack-reused 0
Receiving objects: 100% (174/174), 96.18 MiB | 24.81 MiB/s, done.
Resolving deltas: 100% (79/79), done.
Updating files: 100% (28/28), done.


In [ ]:
cd FaceSwap_DAI

/content/FaceSwap_DAI


###Download_Series

In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/download_1.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/0.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/0.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/download_2.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/1.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/1.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/download_3.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/2.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/2.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/download_4.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/3.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/3.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/download_5.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/4.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/4.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/download_6.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/5.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/5.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



###Images_Series

In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/6.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/6.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_1.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/7.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/7.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_2.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/8.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/8.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_3.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/9.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/9.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_4.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/10.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/10.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_5.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/11.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/11.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_6.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/12.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/12.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_7.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/13.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/13.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_8.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/14.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/14.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_9.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/15.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/15.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_10.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/17.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/17.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_11.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/18.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/18.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_12.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/21.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/21.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_13.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/22.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/22.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_14.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/23.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/23.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_15.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/24.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/24.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_37.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/00.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/00.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_38.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/01.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/01.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_39.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/02.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/02.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_40.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/03.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/03.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_41.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/04.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/04.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_42.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/05.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/05.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/06.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/06.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



###Only_Numbers

In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_16.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/20230226_000918.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/20230226_000918.jpg --correct_color --out_folder False --src_folder False

Invalid SOS parameters for sequential JPEG
qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_17.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/20230226_000945.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/20230226_000945.jpg --correct_color --out_folder False --src_folder False

Invalid SOS parameters for sequential JPEG
qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



###IMG_series

In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_18.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/IMG-20230224-WA0045.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/IMG-20230224-WA0045.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_19.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/IMG-20230224-WA0046.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/IMG-20230224-WA0046.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_20.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/IMG-20230224-WA0047.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/IMG-20230224-WA0047.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_21.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/IMG-20230224-WA0048.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/IMG-20230224-WA0048.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_22.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/IMG-20230224-WA0049.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/IMG-20230224-WA0049.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_23.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/IMG-20230224-WA0051.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/IMG-20230224-WA0051.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_24.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/IMG-20230224-WA0056.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/IMG-20230224-WA0056.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



### WIN_series

In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_25.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/WIN_20230225_23_59_03_Pro.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/WIN_20230225_23_59_03_Pro.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_26.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/WIN_20230225_23_59_15_Pro.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/WIN_20230225_23_59_15_Pro.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_27.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/WIN_20230225_23_59_26_Pro.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/WIN_20230225_23_59_26_Pro.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_28.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/WIN_20230226_00_00_07_Pro.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/WIN_20230226_00_00_07_Pro.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_29.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/WIN_20230226_00_00_30_Pro.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/WIN_20230226_00_00_30_Pro.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_30.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/WIN_20230226_00_00_34_Pro.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/WIN_20230226_00_00_34_Pro.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_31.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/WIN_20230226_00_00_39_Pro.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/WIN_20230226_00_00_39_Pro.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_32.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/WIN_20230226_00_00_49_Pro.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/WIN_20230226_00_00_49_Pro.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_33.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/WIN_20230226_00_30_39_Pro.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/WIN_20230226_00_30_39_Pro.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_34.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/WIN_20230226_00_30_50_Pro.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/WIN_20230226_00_30_50_Pro.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_35.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/WIN_20230226_00_31_15_Pro.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/WIN_20230226_00_31_15_Pro.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



In [ ]:
!python main.py --src /content/drive/MyDrive/DAI_Assignment_1/Q2/FaceSweep_Images/images_36.jpg --dst /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Real/WIN_20230226_00_31_20_Pro.jpg --out /content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset/Fake/WIN_20230226_00_31_20_Pro.jpg --correct_color --out_folder False --src_folder False

qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.8/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.



## Model Building

In [ ]:
transform_face = transforms.Compose([transforms.ToTensor(),transforms.Resize((224,224))])
data_class = datasets.ImageFolder(root='/content/drive/MyDrive/DAI_Assignment_1/Q2/Dataset', transform = transform_face)
train_dataset, val_dataset = random_split(data_class, (50, 50))
train_loader = DataLoader(dataset=train_dataset, shuffle=True, batch_size=1)
test_loader = DataLoader(dataset=val_dataset, shuffle=True, batch_size=1)

###1st Variety (Clean v/s PGD with SVHN)

In [ ]:
detect_res = Detector_Face('resnet_34')
trainer = pl.Trainer(accelerator='gpu', max_epochs=15, resume_from_checkpoint='/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34_detector/lightning_logs/version_1/checkpoints/epoch=4-step=22895.ckpt', default_root_dir='/content/drive/MyDrive/DAI_Assignment_1/Q2/Models', benchmark=True)
trainer.fit(detect_res, train_loader)
del trainer, detect_res
print(" ")
Model_Resnet = model_return('/content/drive/MyDrive/DAI_Assignment_1/Q2/Models/lightning_logs/version_0/checkpoints/epoch=14-step=23395.ckpt', Detector_Face('resnet_34'))
detect_eval(test_loader, Model_Resnet)
del Model_Resnet

Accuracy ===> 52.0%


###2nd Variety (Clean v/s Jitter with SVHN)

In [ ]:
detect_res = Detector_Face('resnet_34')
trainer = pl.Trainer(accelerator='gpu', max_epochs=15, resume_from_checkpoint='/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34_detector/lightning_logs/version_4/checkpoints/epoch=4-step=22895.ckpt', default_root_dir='/content/drive/MyDrive/DAI_Assignment_1/Q2/Models', benchmark=True)
trainer.fit(detect_res, train_loader)
del trainer, detect_res
print(" ")
Model_Resnet = model_return('/content/drive/MyDrive/DAI_Assignment_1/Q2/Models/lightning_logs/version_1/checkpoints/epoch=14-step=23395.ckpt', Detector_Face('resnet_34'))
detect_eval(test_loader, Model_Resnet)
del Model_Resnet

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34_detector/lightning_logs/version_4/checkpoints/epoch=4-step=22895.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | ResNet           | 21.3 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
21.3 M    Trainable params
0         Non-trainable params
21.3 M    Total params
85.143    Total estimated model params size (MB)
INF

Training: 4579it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=15` reached.


 
Accuracy ===> 48.0%


###3rd Variety (Clean v/s PGD with CIFAR10)

In [ ]:
detect_res = Detector_Face('resnet_34')
trainer = pl.Trainer(accelerator='gpu', max_epochs=15, resume_from_checkpoint='/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34_detector_cifar/lightning_logs/version_3/checkpoints/epoch=4-step=15625.ckpt', default_root_dir='/content/drive/MyDrive/DAI_Assignment_1/Q2/Models', benchmark=True)
trainer.fit(detect_res, train_loader)
del trainer, detect_res
print(" ")
Model_Resnet = model_return('/content/drive/MyDrive/DAI_Assignment_1/Q2/Models/lightning_logs/version_2/checkpoints/epoch=14-step=16125.ckpt', Detector_Face('resnet_34'))
detect_eval(test_loader, Model_Resnet)
del Model_Resnet

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34_detector_cifar/lightning_logs/version_3/checkpoints/epoch=4-step=15625.ckpt
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:338: UserWarning: The dirpath has changed from '/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34_detector_cifar/lightning_logs/version_3/checkpoints' to '/content/drive/MyDrive/DAI_Assignment_1/Q2/Models/lightning_logs/version_2/checkpoints', therefore `best_model_score`, `kth_best_model_path`, `kth_value`, `last_model_path` and `best_k_models` won't be reloaded. Only

Training: 3125it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=15` reached.


 
Accuracy ===> 50.0%


###4th Variety (Clean v/s Jitter with CIFAR10)

In [ ]:
detect_res = Detector_Face('resnet_34')
trainer = pl.Trainer(accelerator='gpu', max_epochs=15, resume_from_checkpoint='/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34_detector_cifar/lightning_logs/version_4/checkpoints/epoch=4-step=15625.ckpt', default_root_dir='/content/drive/MyDrive/DAI_Assignment_1/Q2/Models', benchmark=True)
trainer.fit(detect_res, train_loader)
del trainer, detect_res
print(" ")
Model_Resnet = model_return('/content/drive/MyDrive/DAI_Assignment_1/Q2/Models/lightning_logs/version_3/checkpoints/epoch=14-step=16125.ckpt', Detector_Face('resnet_34'))
detect_eval(test_loader, Model_Resnet)
del Model_Resnet

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34_detector_cifar/lightning_logs/version_4/checkpoints/epoch=4-step=15625.ckpt
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:338: UserWarning: The dirpath has changed from '/content/drive/MyDrive/DAI_Assignment_1/Q1/resnet34_detector_cifar/lightning_logs/version_4/checkpoints' to '/content/drive/MyDrive/DAI_Assignment_1/Q2/Models/lightning_logs/version_3/checkpoints', therefore `best_model_score`, `kth_best_model_path`, `kth_value`, `last_model_path` and `best_k_models` won't be reloaded. Only

Training: 3125it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=15` reached.


 
Accuracy ===> 48.0%


#Question-3
###Drive Link--->(https://drive.google.com/drive/folders/1lMVAN3glfGllGXQzQODKoH5U_QdEQ3n-?usp=sharing)

In [ ]:
with open('/content/drive/MyDrive/DAI_Assignment_1/Q3/Recorded/Eng_1000_20230228_040459.json') as f:
    json_data = json.load(f)

with open('/content/drive/MyDrive/DAI_Assignment_1/Q3/Recorded/Eng_1000_20230228_040459.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["wave", "text"])
    for item in json_data:
        writer.writerow([item["wave"], item["text"]])

In [ ]:
with open('/content/drive/MyDrive/DAI_Assignment_1/Q3/Recorded/Hindi_1000_20230301_110259.json') as f:
    json_data = json.load(f)

with open('/content/drive/MyDrive/DAI_Assignment_1/Q3/Recorded/Hindi_1000_20230301_110259.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["wave", "text"])
    for item in json_data:
        writer.writerow([item["wave"], item["text"]])

In [ ]:
with open('/content/drive/MyDrive/DAI_Assignment_1/Q3/random_genrate_english.txt', 'r') as infile:
    with open('/content/drive/MyDrive/DAI_Assignment_1/Q3/Eng_1000.txt', 'w') as outfile:
        for i in tqdm(range(1000)):
            line = infile.readline().strip()
            while line.endswith('\\'):
                line = line[:-1] + infile.readline().strip()
            outfile.write(line + '\n')
infile.close()
outfile.close()

100%|██████████| 1000/1000 [00:00<00:00, 1840.78it/s]


In [ ]:
with open('/content/drive/MyDrive/DAI_Assignment_1/Q3/random_genrate_hindi.txt', 'r') as infile:
    with open('/content/drive/MyDrive/DAI_Assignment_1/Q3/Hindi_1000.txt', 'w') as outfile:
        for i in tqdm(range(1000)):
            line = infile.readline().strip()
            while line.endswith('\\'):
                line = line[:-1] + infile.readline().strip()
            outfile.write(line + '\n')
infile.close()
outfile.close()

100%|██████████| 1000/1000 [00:00<00:00, 2259.25it/s]


In [ ]:
with open('/content/drive/MyDrive/DAI_Assignment_1/Q3/Eng_1000.txt', 'r') as f:
    for i, line in tqdm(enumerate(f)):
        tts = gTTS(line)
        tts.save(f'/content/drive/MyDrive/DAI_Assignment_1/Q3/Generated_English/{i}.wav')

998it [05:58,  2.78it/s]


gTTSError: ignored

In [ ]:
with open('/content/drive/MyDrive/DAI_Assignment_1/Q3/Hindi_1000.txt', 'r', encoding='utf-8') as f:
    for i, line in tqdm(enumerate(f)):
        tts = gTTS(line, lang='hi')
        tts.save(f'/content/drive/MyDrive/DAI_Assignment_1/Q3/Generated_Hindi/{i}.wav')

1000it [07:05,  2.35it/s]
